**Práctica Integrada: Pipeline de Preprocesamiento para Análisis de Feedback en E-Commerce**  
**Contexto Empresarial Real:**  
Eres parte del equipo de NLP de **MarketMind**, una empresa de e-commerce que analiza millones de comentarios de clientes (inglés/español) y tweets para detectar tendencias de productos. **Problema actual:** El modelo GPT-4 fine-tuned genera resúmenes erróneos porque el texto crudo contiene URLs, precios, y emoticones mal normalizados.  

---

### **Fase 1: Limpieza Contextual de Texto**  
**Objetivo:** Crear una función que procese texto crudo preservando información crítica para análisis comercial.  

#### **Datos de Entrada:**  

Archivo -> validation_samples.json

#### **Requisitos:**  
1. **Eliminar:**  
   - URLs, handles de usuario (@MariaP), y signos de puntuación *excepto* `!`, `?`, `%`, `$`, `/`.  
   - Paréntesis `()` y corchetes `[]`.  
2. **Preservar:**  
   - Emoticones (😃, 🔥), hashtags (#ModaDeportiva2023), y formatos comerciales (`2x`, `$99.99`).  
   - Fechas (`30/11/2023`), horas (`18:30`), y porcentajes (`15%`).  
3. **Normalizar:**  
   - Unificar espacios múltiples y saltos de línea.  

#### **Pistas de Implementación:**  
- Usar regex con grupos capturadores para fechas (`\b\d{2}/\d{2}/\d{4}\b`).  
- Para emoticones, usar la librería `emoji` (no eliminar 🚫→ ✅).  
- Preservar `$`, `%`, y `/` solo si están adyacentes a números: `\$?\d+(\.\d+)?%?`.  

#### **Verificación:**  
El texto procesado debe quedar:  
```  
🔥 OFERTA Compre 2x zapatos Nike a $99.99 antes $150 👟  
Válido hasta el 30/11/2023 Atención ¿Envío gratis 😃 #ModaDeportiva2023  
```  

In [7]:
import re
import json
import emoji
import random

def clean_text(text):
    """Función de limpieza de texto para análisis de feedback en e-commerce.
    
    Parámetros:
    text (str): Texto a limpiar
    
    Returns:
    str: Texto limpio y normalizado
    """
    # 1. Identificar y marcar elementos a preservar
    
    # Patrones para elementos a preservar
    patterns_to_preserve = [
        # Hashtags
        (r'(#\w+)', r' \1 '),
        
        # Fechas en formato DD/MM/YYYY o YYYY-MM-DD
        (r'(\b\d{2}/\d{2}/\d{4}\b|\b\d{4}-\d{2}-\d{2}\b)', r' \1 '),
        
        # Horas en formato HH:MM
        (r'(\b\d{1,2}:\d{2}\b)', r' \1 '),
        
        # Formatos comerciales: 2x, $99.99, 15%, etc.
        (r'(\b\d+x\b)', r' \1 '),  # Preservar formatos como 2x
        (r'(\$?\d+(\.\d+)?(€|USD)?)', r' \1 '),  # Preservar precios
        (r'(\d+(\.\d+)?%)', r' \1 ')  # Preservar porcentajes
    ]
    
    # Aplicar patrones de preservación
    for pattern, replacement in patterns_to_preserve:
        text = re.sub(pattern, replacement, text)
    
    # 2. Eliminar elementos no deseados
    
    # Eliminar URLs
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    
    # Eliminar handles de usuario (@usuario)
    text = re.sub(r'@\w+', ' ', text)
    
    # Eliminar paréntesis y corchetes pero preservar su contenido
    text = re.sub(r'[\(\)\[\]{}]', ' ', text)
    
    # 3. Crear una lista de caracteres a mantener (emojis y puntuación permitida)
    chars_to_keep = ['!', '?', '%', '$', '/', '€']
    
    # 4. Procesar el texto caracter por caracter
    result = []
    for char in text:
        # Mantener emojis
        if char in emoji.EMOJI_DATA:
            result.append(char)
        # Mantener puntuación permitida
        elif char in chars_to_keep:
            result.append(char)
        # Mantener letras, números y espacios
        elif char.isalnum() or char.isspace():
            result.append(char)
        # Reemplazar otra puntuación con espacio
        else:
            result.append(' ')
    
    # 5. Normalizar espacios múltiples y saltos de línea
    processed_text = ''.join(result)
    processed_text = re.sub(r'\s+', ' ', processed_text).strip()
    
    return processed_text

# Cargar los datos de ejemplo para pruebas
with open('validation_samples.json', 'r', encoding='utf-8') as f:
    samples = json.load(f)

# Seleccionar un ejemplo aleatorio para verificación
random_sample = random.choice(samples)
ejemplo = random_sample['input']

print("== Verificación Aleatoria ==")
print("\nTexto original:")
print(ejemplo)
print("\nTexto limpio:")
print(clean_text(ejemplo))

== Verificación Aleatoria ==

Texto original:
Received package late. Tracking number: #123ABC. Contacted @support, no reply 😡.

Texto limpio:
Received package late Tracking number 123 ABC Contacted no reply 😡
